In [85]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index import LLMPredictor, PromptHelper, SimpleDirectoryReader, GPTSimpleVectorIndex, Document, GPTListIndex
from langchain import OpenAI
from langchain.vectorstores import  Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from time import sleep
from tqdm import tqdm
import pinecone
import uuid
import json
import openai

embed_model = "text-embedding-ada-002"

In [16]:
# 拉取YouTube上的transcript
def loadYoutubeData(url):
    loader = YoutubeLoader.from_youtube_url(
        url, add_video_info=True)
    data = loader.load()
    # save_file('data/%s.json' % data[0].metadata['title'], data[0].page_content)

    return data


In [18]:
# doc = Document(result[0].page_content.replace("\n", " "))
# txt = result[0].page_content.replace("\n", " ")
print (f'You have {len(result)} document(s) in your data')
print (f'There are {len(result[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 20362 characters in your document


In [78]:
video_url= "https://www.youtube.com/watch?v=ziFipUmd7Mg"#My Time At Portia
#result = loadYoutubeData("https://www.youtube.com/watch?v=-aYsmnlET-Q&list=PLJAzFcYKyx4RX5JBq_rXbQNmUdc9Ugbv1")#wulong
# result = loadYoutubeData("https://www.youtube.com/watch?v=V56uazDSYcA")#Animal Crossing:
# result = loadYoutubeData("https://www.youtube.com/watch?v=hI99HvOHCLI")#Star valley

result = loadYoutubeData(video_url)

chunk_size= 1000#chunk大小 字符数
chunk_overlap = 100#chunk重叠 字符数
video_title = result[0].metadata['title'].replace(" ", "") #标题
video_date= result[0].metadata['publish_date'].strftime("%Y-%m-%d")#日期

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=100)#分割文档
texts = text_splitter.split_documents(result)#分割文档

#构建新数据
new_data=[{ 'id':str(uuid.uuid4()),'text': x.page_content.replace("\n", " "),'title':x.metadata['title'] , 'author': x.metadata['author'],'publish_date':video_date,'chunk_size':chunk_size,'chunk_overlap':chunk_overlap, 'thumbnail_url':x.metadata['thumbnail_url'],'url':video_url } for x in texts]

#储存新数据
json_str = json.dumps(new_data)
with open(f'data/{video_title}.json', 'w') as f:
    f.write(json_str)

print(new_data[0:2])

[{'id': 'f745e13c-f141-4890-8ed3-20dc0dd994c6', 'text': "hey guys josh here and on may 26 2022 my time at cenrock the sequel to my time at porsche finally released in early access on steam this is a game i've been looking forward to for quite some time now it had been delayed numerous times as the developer patiai games had to wait for certification from their government before releasing the game but now everything is in order it is finally available for everyone to play so i would like to tell you what i think of this game so far and also how it compares to its predecessor patea games kindly sent me a review key for the game about 3 weeks ago so i've played a fair bit already keep in mind the game is still in early access so this is not a full review as some features are still incomplete and there may be a bit more bugs than there will be in the full release but hopefully this should give you a pretty good idea of how the game is like and if it's for you so let's start with the story 

In [80]:
import pinecone
index_name = 'game-reviews'

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key="b32bb33f-57b0-47f5-8669-949c9f6f2f4b",
    environment="us-west1-gcp"  # may be different, check at app.pinecone.io
)
# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 23}},
 'total_vector_count': 23}

In [88]:
batch_size = 5  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(new_data), batch_size)):
    # find end of batch
    i_end = min(len(new_data), i+batch_size)
    meta_batch = new_data[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    # try:
    res = openai.Embedding.create(input=texts, engine=embed_model)
    # except:
    #     done = False
    #     while not done:
    #         sleep(5)
    #         try:
    #             res = openai.Embedding.create(input=texts, engine=embed_model)
    #             done = True
    #         except:
    #             pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'author': x['author'],
        'chunk_overlap': x['chunk_overlap'],
         'chunk_size': x['chunk_size'],
        'title': x['title'],
        'text': x['text'],
        'url': x['url'],
        'publish_date': x['publish_date'],
        'thumbnail_url': x['thumbnail_url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)


100%|██████████| 5/5 [00:06<00:00,  1.27s/it]


In [91]:
query = "what are the game mechanic and key features of this game?"
xq = openai.Embedding.create(input=query, engine=embed_model)['data'][0]['embedding']
res = index.query([xq], top_k=3, include_metadata=True)

{'matches': [{'id': '4e139723-38d9-4fab-9dfe-ceb8e87bae6d',
              'metadata': {'text': 'feels smoother the animations are better '
                                   'and you can unlock a few simple combos for '
                                   'each weapon type when you level up your '
                                   'combat skills they also added a break '
                                   'mechanic where if you hit an enemy '
                                   'multiple times in a row they will get '
                                   'stunned for a few seconds and you can also '
                                   'dodge you can roll just like you could in '
                                   'porsche but once again this time around it '
                                   'just feels a bit smoother and all of these '
                                   'things combined make the combat feel much '
                                   'better than in the past also not exact

In [97]:
for match in res['matches']:
    print(f"{match['score']:.3f}: {match['metadata']['text']}")

0.808: feels smoother the animations are better and you can unlock a few simple combos for each weapon type when you level up your combat skills they also added a break mechanic where if you hit an enemy multiple times in a row they will get stunned for a few seconds and you can also dodge you can roll just like you could in porsche but once again this time around it just feels a bit smoother and all of these things combined make the combat feel much better than in the past also not exactly combat related but one more thing i would like to mention is that while in porsche you can only sprint for a little while before your character got exhausted in send rock you can sprint continuously and there's even an option letting you choose if you want to sprint by holding the button or just toggling it once and this makes walking or sprinting between the different areas a lot more fun and a lot faster fast travel is also unlocked a lot faster in this game so there are stops located all around s

In [30]:
docs


[Document(page_content="feels smoother the animations are better and you can unlock a few simple combos for each weapon type when you level up your combat skills they also added a break mechanic where if you hit an enemy multiple times in a row they will get stunned for a few seconds and you can also dodge you can roll just like you could in porsche but once again this time around it just feels a bit smoother and all of these things combined make the combat feel much better than in the past also not exactly combat related but one more thing i would like to mention is that while in porsche you can only sprint for a little while before your character got exhausted in send rock you can sprint continuously and there's even an option letting you choose if you want to sprint by holding the button or just toggling it once and this makes walking or sprinting between the different areas a lot more fun and a lot faster fast travel is also unlocked a lot faster in this game so there are stops loc